In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import io
from tempfile import mkdtemp
import pickle as pkl
import tarfile
import scipy
import time

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# RBF Kernel PCA in Tensorflow

In [0]:
# RBF Kernel PCA
# reference:https://towardsdatascience.com/kernel-pca-vs-pca-vs-ica-in-tensorflow-sklearn-60e17eb15a64
def rbf_kernel_pca(X, gamma, n_components):
    """
    RBF kernel PCA implementation.    
    Parameters
    ------------
    X: {NumPy ndarray}, shape = [n_examples, n_features]  
    gamma: float
        Tuning parameter of the RBF kernel    
    n_components: int
        Number of principal components to return    
    Returns
    ------------
    top_eigvec: {NumPy ndarray}, shape = [n_examples, k_features]   
    """
    # Calculate pairwise squared Euclidean distances
    # Non-loop distance calculation: https://medium.com/@souravdey/l2-distance-matrix-vectorization-trick-26aa3247ac6c
    distance_matric = -2. * tf.matmul(X, tf.transpose(X)) + \
                      tf.reduce_sum(X**2, axis = 1) + \
                      tf.expand_dims(tf.reduce_sum(X**2, axis = 1),1)
    # Compute the symmetric kernel matrix.
    k = tf.exp(-gamma * distance_matric)
    # Center the kernel matrix.
    N = k.shape[0]
    # ones = tf.ones([N,N], dtype = tf.float64)/N
    ones = tf.ones([N,N], dtype = tf.float64)/tf.cast(N, 'float64')
    center_k = k - tf.matmul(ones, k) - tf.matmul(k, ones)+tf.matmul(tf.matmul(ones,k),ones)
    # Obtaining eigenpairs from the centered kernel matrix
    eigval, eigvec = tf.linalg.eigh(center_k)
    # Collect the top k eigenvectors (projected examples)
    # top_eigvec = tf.stack((eigvecs[:,-i] for i in range(1,n_components+1)))
    top_eigvec = eigvec[:, -n_components:]
    lambdas = eigval[-n_components:]
    # lambdas = [eigval[-i] for i in range(1,n_components+1)]
    return top_eigvec, lambdas, eigvec

In [0]:
def project_x(x_new, X, gamma, alphas, lambdas):
    pair_dist = np.array([np.sum((x_new-row)**2) for row in X])
    k = np.exp(-gamma * pair_dist)
    return k.dot(alphas / lambdas)

KPCA Reconstruction Error:

References:
* https://alex.smola.org/papers/1998/SchMikSmoRatetal98.pdf

In [0]:
def reconstruction_error(X, top_eigvecs, top_eigvals):
    '''
    compute reconstruction error for KPCA?
    Args:
      X: {NumPy ndarray}, shape = [n_examples, n_features] 
      top_eigvecs: {NumPy ndarray {ndarray}}, shape = [n_components, k_features]  
      top_eigvals: {NumPy ndarray {ndarray}}, shape = [n_components, k_features]  
    Returns:
    error:(float) reconstruction error
    '''
    error = 0.0

    # project every data point onto subspace using the principal components


    # compute distance (i.e. error) from original data point to its projection

    
    
    return error

### Running on pickled data

In [0]:
fp = np.memmap("/content/drive/My Drive/train_img_array.pkl", dtype = 'float32', mode = 'r', shape = (1803460,32,32))

In [0]:
fp = fp.reshape((1803460,-1))

In [0]:
data_tensor = tf.convert_to_tensor(fp, dtype = tf.float64)

In [0]:
import gc
gc.collect()

0

In [0]:
start_time = time.time()
kpca_x, lambdas, eigvec = rbf_kernel_pca(data_tensor[:100000], 10, 128)
end_time  = time.time()
print('Orignial dimension: %d' % (data_tensor.shape[1]))
print("handling %d data: --- {%s} seconds ---" % (data_tensor.shape[0], time.time() - start_time))

## Testing on image data

In [0]:
# test small amount of images
TEST_NUM_IMAGES = 200

data = []
tar_file = tarfile.open("/content/drive/My Drive/data_val_data.tar")

num = 0
for member in tar_file.getmembers():
    if num <= TEST_NUM_IMAGES:
        f = tar_file.extractfile(member)
        if f is not None:
            content = f.read()
            data.append(content)
        num += 1
    else:
        break
tar_file.close()

In [0]:
def convert_img(img_data, dim_h):
    """ 
  Resize image so that it has height dim_h and flatten the image
  Args:
    img_data: (bytes) image data
    img_dim: typle(width, height)
  Returns:
    img: (np array) the resized and flattened image
    """
    img = Image.open(io.BytesIO(img_data))
    img = img.convert(mode = 'L') # convert to grey scale
    img = img.resize((dim_h, dim_h), Image.ANTIALIAS)
    im_flatten = np.reshape(np.array(img),(-1,))
    return im_flatten/255

# numpy memmap

In [0]:
filename = 'flatten_array.pkl'
fp = np.memmap(filename, dtype='float32', mode='w+', shape=(len(data),32*32))

In [0]:
# convert 100 image at once
start_time = time.time()
for i in range(len(data)//100):

    start, end = i*100, min((i+1)*100, len(data))
    array = np.array([convert_img(i, 32) for i in data[start:end]])
    fp[start:end] = array
end_time  = time.time()
print("Loading {%d} data: --- {%s} seconds ---" % (len(data), time.time() - start_time))

Loading {200} data: --- {1.5101280212402344} seconds ---


In [0]:
data_tensor = tf.convert_to_tensor(fp, dtype = tf.float64)

### Test KPCA on different size of data

In [0]:
start_time = time.time()
kpca_x, lambdas = rbf_kernel_pca(data_tensor, 10, 128)
end_time  = time.time()
print('Orignial dimension: %d' % (data_tensor.shape[1]))
print("handling %d data: --- {%s} seconds ---" % (data_tensor.shape[0], time.time() - start_time))

Orignial dimension: 1024
handling 1000 data: --- {4.321377754211426} seconds ---


In [0]:
import gc
gc.collect()

0

In [0]:
start_time = time.time()
kpca_x, lambdas = rbf_kernel_pca(data_tensor, 10, 128)
end_time  = time.time()
print('Orignial dimension: %d' % (data_tensor.shape[1]))
print("handling %d data: --- {%s} seconds ---" % (data_tensor.shape[0], time.time() - start_time))

Orignial dimension: 1024
handling 5000 data: --- {4.290626764297485} seconds ---


In [0]:
start_time = time.time()
kpca_x, lambdas = rbf_kernel_pca(data_tensor, 10, 128)
end_time  = time.time()
print('Orignial dimension: %d' % (data_tensor.shape[1]))
print("handling %d data: --- {%s} seconds ---" % (data_tensor.shape[0], time.time() - start_time))

Orignial dimension: 1024
handling 10000 data: --- {26.140833854675293} seconds ---


In [0]:
# using TPU
start_time = time.time()
kpca_x, lambdas = rbf_kernel_pca(data_tensor, 10, 128)
end_time  = time.time()
print('Orignial dimension: %d' % (data_tensor.shape[1]))
print("handling %d data: --- {%s} seconds ---" % (data_tensor.shape[0], time.time() - start_time))

Orignial dimension: 1024
handling 20000 data: --- {9445.67579293251} seconds ---


In [0]:
start_time = time.time()
kpca_x, lambdas = rbf_kernel_pca(data_tensor, 10, 128)
end_time  = time.time()
print('Orignial dimension: %d' % (data_tensor.shape[1]))
print("handling %d data: --- {%s} seconds ---" % (data_tensor.shape[0], time.time() - start_time))

Orignial dimension: 4096
handling 1000 data: --- {4.342210292816162} seconds ---


In [0]:
start_time = time.time()
kpca_x, lambdas = rbf_kernel_pca(data_tensor, 10, 128)
end_time  = time.time()
print('Orignial dimension: %d' % (data_tensor.shape[1]))
print("handling %d data: --- {%s} seconds ---" % (data_tensor.shape[0], time.time() - start_time))

Orignial dimension: {4096}
handling {5000} data: --- {5.550684690475464} seconds ---


In [0]:
start_time = time.time()
kpca_x, lambdas = rbf_kernel_pca(data_tensor, 10, 128)
end_time  = time.time()
print('Orignial dimension: %d' % (data_tensor.shape[1]))
print("handling %d data: --- {%s} seconds ---" % (data_tensor.shape[0], time.time() - start_time))

Orignial dimension: 4096
handling 10000 data: --- {7.483554840087891} seconds ---


Start using TPU now.

In [0]:
import gc
gc.collect()

417

In [0]:
start_time = time.time()
kpca_x, lambdas = rbf_kernel_pca(data_tensor, 10, 128)
end_time  = time.time()
print('Orignial dimension: %d' % (data_tensor.shape[1]))
print("handling %d data: --- {%s} seconds ---" % (data_tensor.shape[0], time.time() - start_time))

Orignial dimension: 4096
handling 20000 data: --- {12942.332182884216} seconds ---


In [0]:
start_time = time.time()
kpca_x, lambdas, egivec = rbf_kernel_pca(data_tensor, 10, 128)
end_time  = time.time()
print('Orignial dimension: %d' % (data_tensor.shape[1]))
print("handling %d data: --- {%s} seconds ---" % (data_tensor.shape[0], time.time() - start_time))

Orignial dimension: 1024
handling 200 data: --- {0.0198516845703125} seconds ---


In [0]:
kpca_x.shape

TensorShape([Dimension(200), Dimension(128)])

In [0]:
start_time = time.time()
alphas, lambdas, eigvec = rbf_kernel_pca(data_tensor, 10, 128)
end_time  = time.time()
print('Orignial dimension: %d' % (data_tensor.shape[1]))
print("handling %d data: --- {%s} seconds ---" % (data_tensor.shape[0], time.time() - start_time))

Orignial dimension: 1024
handling 200 data: --- {0.017661333084106445} seconds ---


### Build SVM model

In [0]:
# read label file
tar_file = tarfile.open("/content/drive/My Drive/data_filelist_places365-standard.tar")
f = tar_file.extractfile("places365_val.txt")
content = f.read().decode(encoding="utf-8")

In [0]:
val_label = tf.convert_to_tensor(np.array([int(i.split(' ')[1]) for i in content.split('\n')]), dtype = tf.float64)

In [0]:
val_label.shape

TensorShape([Dimension(36500)])

In [0]:
svm = SVM(data_tensor, val_label[:200], 0.2)

In [0]:
svm = svm_model(3, 200)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
svm.fit(data_tensor, val_label[:200])

KeyboardInterrupt: ignored

In [0]:
svm.fit(data_tensor,val_label[:200])

In [0]:
import operator as op
from functools import reduce

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
class svm_model:
    def __init__(self, n_class, dimension, learning_rate=1e-2, regularization=1):
        self.learning_rate=learning_rate
        self.n_class=n_class
        self.dimension=dimension
        self.w=[]
        self.b=[]
        self.model_y=[]
        self.loss=[]
        self.hinge_loss=[]
        self.regularization_loss=[]
        #prediction=[]
        self.gt_y=[]
        self.lookup_class=dict()
        self.w_model=np.random.normal(loc=0,scale=1,size=(self.ncr(n_class,2), dimension))
        self.b_model=np.random.normal(loc=0,scale=1,size=(self.ncr(n_class,2),1))
        self.lookup_matrix=np.zeros((n_class, self.ncr(n_class,2)),dtype=np.float32)
        self.batch_x=tf.placeholder(tf.float32,shape=(None,dimension),name="batch_x")
        self.batch_y=tf.placeholder(tf.float32,shape=(None,1),name="batch_y")
        self.w = tf.Variable(tf.random_uniform([self.ncr(n_class,2), self.dimension]))
        self.b = tf.Variable(tf.random_uniform([self.ncr(n_class,2),1]))
        k=0
        for i in range(n_class-1):
            for j in range(i+1,n_class):
                self.lookup_class[k]=[i,j]
                k += 1

        for i in range(n_class):
            for j in range(self.ncr(n_class,2)):
                if i in self.lookup_class[j]:
                    if self.lookup_class[j][0]==i:
                        self.lookup_matrix[i,j]=1.0
                    else:
                        self.lookup_matrix[i,j]=-1.0

        for i in range(self.ncr(n_class,2)):
            # idx is the index of all the samples svm i concerns
            
            # idx=tf.where[condition is true] 
            # tf.gather_nd(a,idx) 
            # is equivalent to a[condition is true]
            idx=tf.where(tf.keras.backend.any(tf.equal(self.batch_y,self.lookup_class[i]),1)) 
            
            # 1 x N matrix
            self.model_y.append(
                tf.tanh(
                    tf.matmul(tf.reshape(self.w[i,:],(1,dimension)),
                              tf.gather_nd(self.batch_x, idx), transpose_b=True)
                    + self.b[i,:])) 
            
            # 1 x N matrix
            self.gt_y.append(tf.reshape(self.zonp(tf.cast(
                tf.equal(tf.gather_nd(self.batch_y, idx),self.lookup_class[i][0]),
                tf.float32)),(1,-1)))
            
            self.hinge_loss.append(tf.reduce_mean(tf.maximum(0.0,1-tf.multiply(self.model_y[i],self.gt_y[i]))))
            self.regularization_loss.append(regularization * tf.norm(self.w[i,:], 2))
            self.loss.append(self.hinge_loss[i] + self.regularization_loss[i])
        self.total_loss = sum(self.loss)
        self.opt = tf.train.RMSPropOptimizer(learning_rate).minimize(self.total_loss)
        
    def ncr(self,n, r):
        r = min(r, n-r)
        numer = reduce(op.mul, range(n, n-r, -1), 1)
        denom = reduce(op.mul, range(1, r+1), 1)
        return numer // denom
    
    def zonp(self,zero_one):
        return 2*zero_one-1

    def fit(self,data_x,data_y,iter_time=100, batch_size=1000):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            sess.run([tf.assign(self.w,self.w_model),tf.assign(self.b,self.b_model)])
            
            total = data_y.shape[0]
            lower = 0
            upper = lower + batch_size
            
            counter = 0
            while counter < iter_time:
                lower = max(0,        (lower   +batch_size) % total)
                upper = min(total,      (lower +batch_size) % total)
                if lower<upper:
                    _,loss_val = sess.run([self.opt,self.total_loss], feed_dict={self.batch_x:data_x[lower:upper],self.batch_y:data_y[lower:upper]})
                    print("iteration:"+str(counter)+" loss:"+str(loss_val))
                    counter += 1
                
            w_model,b_model = sess.run([self.w,self.b])
        self.w_model = w_model
        self.b_model = b_model
        
        return w_model,b_model
    
    def predict(self,data_x):
        result = np.argmax(np.matmul(self.lookup_matrix, np.tanh(np.matmul(self.w_model,data_x.T)) ),axis=0)
        return result.reshape(-1,1)

In [0]:
class SVM:
    def __init__(self, X, y, reg):
        """ Initialize the SVM attributes and initialize the weights vector to the zero vector. 
            Attributes: 
                X (tensor) : training data intputs
                y (tensor) : 1D numpy array of training data outputs
                reg (float) : regularizer parameter
                theta : 1D tensor of weights
        """
        self.X = X
        self.y = y
        self.reg = reg
        self.theta = tf.zeros([self.X.shape[1], ], dtype = tf.float64)
    
    def objective(self, X, y):
        """ Calculate the objective value of the SVM. When given the training data (self.X, self.y), this is the 
            actual objective being optimized. 
            Args:
                X (array_like) : array of examples, where each row is an example
                y (array_like) : array of outputs for the training examples
            Output:
                (float) : objective value of the SVM when calculated on X,y
        """
        theta_xy = tf.math.multiply(self.y, tf.tensordot(self.X, self.theta, axes = 1))
        hinge = tf.reduce_sum(tf.math.maximum(1-1-theta_xy, 0))
        reg = (self.reg / 2) * scipy.linalg.norm(self.theta) ** 2
        objective = hinge + reg
        
        return objective
    
    def gradient(self):
        """ Calculate the gradient of the objective value on the training examples. 
            Output:
             (vector) : 1D numpy array containing the gradient
        """
        u = tf.tensordot(self.X, self.theta, axes = 1)
        v = tf.math.multiply(self.y, u)
        v = tf.cast(tf.where(v<=1, 1., 0.), 'float64')
        y_v = (-self.y * v)[:,None]
        grad = tf.reshape((tf.transpose(y_v) @ self.X + self.theta * self.reg),[-1])

        return grad
     
    def train(self, niters=100, learning_rate=1, verbose=False):
        """ Train the support vector machine with the given parameters. 
            Args: 
                niters (int) : the number of iterations of gradient descent to run
                learning_rate (float) : the learning rate (or step size) to use when training
                verbose (bool) : an optional parameter that you can use to print useful information (like objective value)
        """
        for epoch in range(niters):
            grad = tf.transpose(self.gradient())
            self.theta = self.theta - (grad * learning_rate)
        return

    def predict(self, X):
        """ Predict the class of each label in X. 
            Args: 
                X (array_like) : array of examples, where each row is an example
            Output:
                (vector) : 1D numpy array containing predicted labels
        """
        predict = tf.tensordot(self.X, self.theta, axes = 1)
        predict = tf.where(predict>0,1.,-1.)
        
        return predict

### MNIST

In [3]:
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# normalizing the data
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# flatten data
x_train_flatten = tf.reshape(x_train,[60000,-1])[:500]

In [6]:
x_train_flatten.shape

TensorShape([500, 784])

In [0]:
top_eigvec, lambdas, eigvec = rbf_kernel_pca(x_train_flatten, 10, 15)

In [10]:
top_eigvec

<tf.Tensor: shape=(500, 15), dtype=float64, numpy=
array([[-7.76240381e-06,  3.75198881e-05,  3.48430923e-03, ...,
        -3.01526888e-03, -2.16075392e-06,  2.83735552e-03],
       [-4.92063565e-06,  9.23051778e-06,  2.89294090e-03, ...,
        -2.55867021e-03,  5.37371281e-06,  2.83054366e-03],
       [ 3.25891103e-06,  7.97367664e-07,  2.91587507e-03, ...,
        -2.58160660e-03,  1.35116206e-06,  2.82731243e-03],
       ...,
       [-3.75496019e-06, -2.48294624e-06,  2.90886946e-03, ...,
        -2.57770129e-03,  5.24235518e-07,  2.83034307e-03],
       [-5.75964428e-06,  1.11944498e-05,  2.69223068e-03, ...,
        -2.45200860e-03, -3.64230128e-06,  2.82711545e-03],
       [ 5.12250251e-05,  2.60798209e-05,  6.18465873e-03, ...,
        -3.67241530e-03, -5.35296457e-06,  2.84987889e-03]])>